# Sorting Reviews

In [1]:
# Yorumlar sıralanırken yorumların düşük veya yüksek puanlı olması ile değil,
# pazar yeri olarak kullaanıcılara en doğru sosyal ispatı ulaştırmaya çalışmaktayız.

# Bu bölümde yorumlarımızı hangi metriklere göre (tarih, puan veya user quality) sıralamamız gerektiğiini açıklayacağız.
# USer quality yorumları faydalı bulma sayısı ile ölçümlenebilir.

In [2]:
import pandas as pd
import math
import scipy.stats as st

### Up-Down Diff Score = (up ratings) − (down ratings)

In [3]:
# Review 1: 600 up 400 down total 1000
# Review 2: 5500 up 4500 down total 10000

def score_up_down_diff(up, down):
    return up - down

In [4]:
# Review 1 Score:
score_up_down_diff(600, 400)

200

In [5]:
# Review 2 Score
score_up_down_diff(5500, 4500)

1000

In [6]:
# Lakin yukarıda Review 1'in yüzdesinin %60 olumlu olduğunu, review 2'nin %55 olumlu olduğunu görüyoruz.
# Farktan dolayı review 2 daha iyi gibi gözükse de yüzdelik olarak Review 1 daha iyidir.

# Burada bir eşik değer olması gerekmektedir.

### Average rating = (up ratings) / (all ratings)

In [7]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [8]:
score_average_rating(600, 400)

0.6

In [9]:
score_average_rating(5500, 4500)

0.55

### Wilson Lower Bound Score

In [10]:
# WLB Score ikili etkileşimler (like-dislike vb.) sonucunda ortaya çıkan ölçme problemlerinde kullanılan bir score yöntemidir.

def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [11]:
wilson_lower_bound(600, 400)

0.5693094295142663

In [12]:
wilson_lower_bound(5500, 4500)

0.5402319557715324

In [13]:
wilson_lower_bound(2, 0)

0.3423802275066531

In [14]:
wilson_lower_bound(100, 1)

0.9460328420055449

### Case Study

In [15]:
up = [15, 70, 14, 4, 2, 5, 8, 37, 21, 52, 28, 147, 61, 30, 23, 40, 37, 61, 54, 18, 12, 68]
down = [0, 2, 2, 2, 15, 2, 6, 5, 23, 8, 12, 2, 1, 1, 5, 1, 2, 6, 2, 0, 2, 2]
comments = pd.DataFrame({"up": up, "down": down})

comments

,up,down
0,15,0
1,70,2
2,14,2
3,4,2
4,2,15
5,5,2
6,8,6
7,37,5
8,21,23
9,52,8


In [16]:
# score_pos_neg_diff
comments["score_pos_neg_diff"] = comments.apply(lambda x: score_up_down_diff(x["up"],
                                                                             x["down"]), axis=1)

# score_average_rating
comments["score_average_rating"] = comments.apply(lambda x: score_average_rating(x["up"], x["down"]), axis=1)

# wilson_lower_bound
comments["wilson_lower_bound"] = comments.apply(lambda x: wilson_lower_bound(x["up"], x["down"]), axis=1)

In [17]:
comments

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,15,0,15,1.000000,0.796117
1,70,2,68,0.972222,0.904258
2,14,2,12,0.875000,0.639772
3,4,2,2,0.666667,0.299993
4,2,15,-13,0.117647,0.032880
5,5,2,3,0.714286,0.358934
6,8,6,2,0.571429,0.325906
7,37,5,32,0.880952,0.749996
8,21,23,-2,0.477273,0.337550
9,52,8,44,0.866667,0.758348


In [18]:
comments.sort_values("wilson_lower_bound", ascending=False)

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
11,147,2,145,0.986577,0.952384
12,61,1,60,0.983871,0.914133
1,70,2,68,0.972222,0.904258
21,68,2,66,0.971429,0.901677
18,54,2,52,0.964286,0.878812
15,40,1,39,0.975610,0.874049
13,30,1,29,0.967742,0.838059
16,37,2,35,0.948718,0.831144
19,18,0,18,1.000000,0.824121
17,61,6,55,0.910448,0.818072
